In [1]:
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
%matplotlib inline
sns.set_theme()

In [2]:
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [3]:
from src.utils import get_config, read_file

In [4]:
config = get_config.read_yaml()
symbols = config['data']['symbols']
symbols

['ETH/USDT',
 'BTC/USDT',
 'BNB/USDT',
 'SOL/USDT',
 'ADA/USDT',
 'LINK/USDT',
 'XRP/USDT',
 'LTC/USDT',
 'XLM/USDT']

In [8]:
data = read_file.read_merged_training_data()
data

Reading merged Training data: merged_training_data !
Failed to read merged_training_data: [Errno 2] No such file or directory: '/Users/aayushsharma/Projects/Project-catRL/data/training/merged/merged_training_data.csv' !


In [6]:
def generate_position(data):
    for symbol in symbols:
        symbol = symbol.split('/')[0]
        col = ('position', symbol)
        data[col] = 0.11
    return data